In [5]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
from datetime import datetime, UTC
from dotenv import load_dotenv

current_dir = Path(os.getcwd()).resolve()
if current_dir.name == "LLMPolReasonEval": # uruchomione w Jupyter Lab
    project_root = current_dir
else:  # uruchomione w PyCharm
    project_root = current_dir.parents[2]
print(f"Project root: {project_root}")

sys.path.append(str(project_root / "src"))
from llm_pol_reason_eval.question_processing.dataset_manager import DatasetManager

if load_dotenv(os.path.join(project_root, '.env')):
    print (f"Loaded environment variables from {project_root / '.env'}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Project root: C:\Users\piotr\PycharmProjects\LLMPolReasonEval
Loaded environment variables from C:\Users\piotr\PycharmProjects\LLMPolReasonEval\.env


In [6]:
dataset_raw_path = project_root / "data" / "dataset_raw"
output_dir = project_root / "data" / "dataset"
output_dir.mkdir(parents=True, exist_ok=True)

In [7]:
# Inicjalizacja managera i ładowanie danych
dataset_manager = DatasetManager()
for json_file in dataset_raw_path.rglob("*.json"):
    if "MPOP-P1-100-2412-gemini25pro-2025-06-14T20-55-00Z.json" in str(json_file):
        print(f"Pomijam plik: {json_file}")
        continue
    dataset_manager.add_data_from_json_file(str(json_file))

# Generowanie ścieżki wyjściowej z timestampem ISO 8601
iso_timestamp = datetime.now(UTC).replace(microsecond=0).isoformat().replace("+00:00", "Z").replace(":", "-")
output_filename = f"polski_matura_dataset_{iso_timestamp}.json"
output_path = output_dir / output_filename

# Zapis datasetu
dataset_manager.save_all_data_to_json_file(str(output_path))
print(f"Zapisano dataset do: {output_path}")

Pomijam plik: C:\Users\piotr\PycharmProjects\LLMPolReasonEval\data\dataset_raw\matury\jezyk_polski\MPOP-P1-100-2412-gemini25pro-2025-06-14T20-55-00Z.json
Zapisano dataset do: C:\Users\piotr\PycharmProjects\LLMPolReasonEval\data\dataset\polski_matura_dataset_2025-06-14T20-00-06Z.json


In [8]:
dataset_manager.get_stats()

{'question_category_stats': [('matura_język_polski', 267)],
 'question_type_stats': [('open_text', 179),
  ('closed_MTF', 28),
  ('closed_MCQ', 16),
  ('open_essay', 13),
  ('open_summary', 10),
  ('open_poetry_interpretation', 8),
  ('closed_MRQ', 7),
  ('open_synthesis', 6)]}